In [8]:
# Script para generar un CSV con el dataset modificado
import pandas as pd
import random

# Función para generar un número de cédula ecuatoriana válido
def generar_cedula():
    provincia = random.randint(1, 24)  # Provincias de Ecuador (1-24)
    digitos = f"{provincia:02d}" + ''.join(str(random.randint(0, 9)) for _ in range(7))
    verificador = random.randint(0, 9)
    return digitos + str(verificador)

# Crear el dataset
usuarios = [
    {"Usuario": f"00{i+1}",
     "Gasto Mensual (M)": random.randint(10, 40),
     "Meses de Racha (R)": random.randint(1, 12),
     "Saldo Promedio (S)": random.randint(50, 300),
     "Perdidas de Racha": random.randint(0, 3),
     "Identificacion": generar_cedula()}
    for i in range(100)
]

# Calcular el factor de penalización acumulativa
def calcular_penalizacion(perdidas):
    return round(0.9 ** perdidas, 3)

for usuario in usuarios:
    usuario["Factor de Penalizacion (P)"] = calcular_penalizacion(usuario["Perdidas de Racha"])

# Crear un DataFrame
df = pd.DataFrame(usuarios)

# Guardar en un CSV
df.to_csv("dataset_sc_penalizacion.csv", index=False)

print("CSV generado exitosamente: dataset_sc_penalizacion.csv")


CSV generado exitosamente: dataset_sc_penalizacion.csv


In [34]:
import pandas as pd
import random

# Cargar el dataset principal
df = pd.read_csv("resumen_rachas.csv")

# Asegurar que el campo 'Usuario' tenga formato de tres dígitos

df["Usuario"] = df["Usuario"].apply(lambda x: f"{int(x):03}")

# Listas para las descripciones y tipos de transacción
descripciones = [
    "Pago de suscripción Spotify",
    "Pago de Uber",
    "Compra en tienda digital",
    "Pago de suscripción Netflix",
    "Compra de videojuego",
    "Pedido de comida",
    "Recarga de celular"
]

tipos_transaccion = ["Débito", "Crédito"]

# Función para generar detalles de rachas
def generar_detalles(row):
    detalles = []
    for i in range(int(row["Meses de Racha (R)"])):
        detalles.append({
            "Usuario": row["Usuario"],
            "Mes": i + 1,
            "Estado": "Ganada",
            "Día del Mes": random.randint(1, 28),
            "Tipo de Transacción": random.choice(tipos_transaccion),
            "Descripción": random.choice(descripciones),
            "Valor": round(random.uniform(0.1, 0.3) * row["Gasto Mensual (M)"], 2)
        })

    for j in range(int(row["Perdidas de Racha"])):
        detalles.append({
            "Usuario": row["Usuario"],
            "Mes": int(row["Meses de Racha (R)"]) + j + 1,
            "Estado": "Perdida",
            "Día del Mes": random.randint(1, 28),
            "Tipo de Transacción": "No Pago",
            "Descripción": "Fallo en el pago",
            "Valor": 0
        })

    return detalles

# Generar los detalles de rachas
detalles = []
for _, row in df.iterrows():
    detalles.extend(generar_detalles(row))

# Crear un DataFrame con los detalles
df_detalles = pd.DataFrame(detalles)

# Guardar en un nuevo CSV
df_detalles.to_csv("detalles_rachas.csv", index=False)

print("CSV generado exitosamente: detalles_rachas.csv")


CSV generado exitosamente: detalles_rachas.csv


In [33]:
import pandas as pd
import random

# Función para generar un número de cédula ecuatoriana válido
def generar_cedula():
    provincia = random.randint(1, 24)  # Provincias de Ecuador (1-24)
    digitos = f"{provincia:02d}" + ''.join(str(random.randint(0, 9)) for _ in range(7))
    verificador = random.randint(0, 9)
    return digitos + str(verificador)

# Crear el dataset
usuarios = [
    {"Usuario": f"00{i+1}",
     "Gasto Mensual (M)": random.randint(10, 40),
     "Meses de Racha (R)": random.randint(1, 12),
     "Saldo Promedio (S)": random.randint(50, 300),
     "Perdidas de Racha": random.randint(0, 3),
     "Identificacion": generar_cedula()}
    for i in range(100)
]

# Calcular el factor de penalización acumulativa
def calcular_penalizacion(perdidas):
    return round(0.9 ** perdidas, 3)

for usuario in usuarios:
    usuario["Factor de Penalizacion (P)"] = calcular_penalizacion(usuario["Perdidas de Racha"])

# Calcular el score de confianza en función de las rachas y el gasto mensual
def calcular_score_confianza(gasto, meses_racha):
    # Estabilidad (Meses de Racha): Un usuario con más rachas tiene mayor confianza
    estabilidad = meses_racha / 12  # Normalizar la racha en un rango de 0 a 1

    # Impacto positivo del Gasto Mensual
    if gasto > 30:
        impacto = 0.3  # Gasto alto aumenta la confianza
    elif gasto < 15:
        impacto = 0.1  # Gasto bajo también aumenta la confianza, pero menos
    else:
        impacto = 0.2  # Gasto moderado tiene un impacto medio

    # Score final, combinando la estabilidad y el impacto positivo del gasto
    score = round((0.6 * estabilidad) + (0.4 * impacto), 2)
    return score

# Agregar el Score de Confianza a cada usuario
for usuario in usuarios:
    usuario["Score de Confianza"] = calcular_score_confianza(usuario["Gasto Mensual (M)"], usuario["Meses de Racha (R)"])

# Agregar el porcentaje del Score de Confianza
for usuario in usuarios:
    usuario["Score de Confianza (%)"] = round(usuario["Score de Confianza"] * 100, 2)

# Calcular la capacidad de pago basada en el Score de Confianza
def calcular_capacidad_pago(saldo_promedio, score_confianza):
    factor_ajuste = 1.5  # Este valor puede ajustarse según sea necesario
    capacidad_pago = round(saldo_promedio * score_confianza * factor_ajuste, 2)
    return capacidad_pago

# Agregar la Capacidad de Pago (C) a cada usuario
for usuario in usuarios:
    usuario["Capacidad de Pago (C)"] = calcular_capacidad_pago(usuario["Saldo Promedio (S)"], usuario["Score de Confianza"])

# Calcular el Crédito a Otorgar basado en la Capacidad de Pago
def calcular_credito(capacidad_pago):
    # Se utiliza un porcentaje fijo sobre la capacidad de pago (ajustable)
    porcentaje_credito = 0.5  # 50% de la capacidad de pago como crédito mensual
    credito_mensual = round(capacidad_pago * porcentaje_credito, 2)
    
    # Determinar el plazo de 6 a 12 meses
    plazo_meses = 12#random.randint(6, 12)
    
    # Calcular el total de crédito a otorgar en base al plazo
    credito_total = round(credito_mensual * plazo_meses, 2)
    
    return credito_total

# Agregar la columna de Crédito a Otorgar (total en 6 a 12 meses)
for usuario in usuarios:
    usuario["Credito a Otorgar (6 Meses)"] = calcular_credito(usuario["Capacidad de Pago (C)"])

# Crear un DataFrame
df = pd.DataFrame(usuarios)

# Guardar en un CSV
df.to_csv("resumen_rachas.csv", index=False)

print("CSV generado exitosamente: dataset_sc_penalizacion_confianza_capacidad_credito_total.csv")


CSV generado exitosamente: dataset_sc_penalizacion_confianza_capacidad_credito_total.csv
